# 基座大模型DeepSeek相关教案

## 一、DeepSeek-R1核心技术

### （1）训练流程，分为四步  

**stage1:SFT-冷启动微调**    
目标是为后续的深度学习RL阶段提供基础，数据规模为千级别的高质量思维链数据，具体作用为稳定后续的深度学习过程，改善模型的输出语言质量，解决多语言表达差异统一格式。

**stage2:RL-推理能力的涌现**  
目标是激发模型的深层推理能力，通过GRPO算法激发模型潜能，使其涌现。还可以训练其推理能力，用需要多步推理的数据。

**stage3:SFT-全面能力的提升**  
该阶段目标是平衡推理能力和通用能力，使用大量高质量（600万）推理性数据和（200万）通用任务数据，保持其涌现效果，回复因为推理而下降的通用语言处理能力，进行全面的优化

**stage4:RL-人类偏好对齐**  
目标是规范模型行为，对齐人类价值观，符合人类规范，然后使用多样化prompt数据确保模型在各种场景下的稳定。

小模型不能通过同样的方式获得涌现能力：只有基础模型具备⾜够知识储备（671B参数规模），强化学习才能有效激发推理能⼒。⼩模型（如32B）因知识容量不⾜，⽆法通过相同策略实现能⼒涌现。

### (2)冷启动的目的与实现

冷启动定义：在DeepSeek-r1训练的第一步，使用千级别的思维链数据，对大模型进行监督微调，为后续的强化学习铺垫稳定基础的过程。

核心目标是：  
稳定后续强化学习的过程，让模型学会怎么使用思维链格式的输出。  
改善模型的输出质量，统一多语言的输出格式

冷启动的重要性在于它是后续训练步骤的基础，稳定的SFT模型是RL训练成功的基础，而且它也为大模型涌现提供了知识储备并建立安全边界

### （3）MOE架构设计

为什么需要MOE：解决参数爆炸问题，突破计算效率的瓶颈。

问题背景：传统架构大模型每一次推理会激活所有参数，但真正有贡献的参数是少数。激活全部参数会导致显存开销巨大，浪费算力。

MOE的解决方案：稀疏激活机制，每一次仅激活少数专家，其余专家休眠。与传统稠密Dense模型相比，显存开销大幅减少。

**DeepSeek的MOE门控训练：**  
门控网络工作原理：将输入token与专家的权重矩阵进行线性变化，计算出每一个token对每一个专家的匹配度，归一化后得到适配分，选择top-k，被选中的专家各自给出答案，然后进行加权输出

**主损失：** 合并结果与正确答案的差距，衡量模型的预测准确性。在更新权重矩阵这一方面：主损失让门控学会“把 token 分配给最能提升任务性能的专家”    
**辅助损失：**    
检查专家调度是否公平      
辅助损失= 专家⼯作量差异× 惩罚系数确保各专家负载均衡，避免专家"极化"    
辅助损失让门控学会“公平分配，避免专家闲置或过载”  

**DeepSeek对比传统MOE的改进：**  
核⼼创新：将专家分为Shared Expert(共享专家）和Routed Expert（路由专家）    
Shared Expert始终保持激活状态，处理通用性知识  
Routed Expert采用传统做法（先计算相似度，再取top-k）  
最后将二者的输出做加权相加

优势在于：减少专家冗余，用共享专家处理通用性知识，路由专家处理差异性知识，分工明确。另外一个优点是，专家一般都分布在不同的GPU设备，计算时会产生通讯时间消耗，使用共享专家提升通信效率，共享专家计算的时候可以进行路由专家的通讯，从而隐藏部分通讯时间，提升GPU利用率。

**DeepSeek MOE的负载均衡：**  
为什么要做负载均衡：防止出现专家极化现象，一部分专家过载，一部分专家闲置，效率低下（训练的其它没被激活的专家浪费了，模型泛化能力差，模型性能没提升，训练不稳定）最终退化成小型稠密模型，失去价值。   

优化策略：使用动态偏置调整（可用于训练或推理阶段，相比辅助损失只能用于训练），显示负载统计，根据这个值自动降低负载过高的专家的偏置，减少被选中的概率。

**门控的训练方法：**  
使用端到端的训练方式从输入到输出，整个模型一体训练，中间所有组件（包括门控）都通过同一个损失函数联合优化。    
结合任务损失和负载均衡损失，通过梯度反向传播优化门控参数。    
**训练技巧：**     
渐进式训练：先训练基础模型，再引入MOE架构（MOE训练不稳定，从0开始训练容易出现专家极化，先用稠密模型学习通用表示能帮助MOE更好的分工）  
温度调度：调整softmax温度控制专家选择的确定性  
正则化：添加稀疏性正则化防止过度依赖少数专家  

### （4）FP8混合训练

FP8精度是专门为AI训练设计的一种格式，与常用精度对比：    
FP32：占⽤32位，范围约±10³⁸，精度约7位有效数字   
FP16：占⽤16位，范围约±10⁵，精度约3-4位有效数字  
FP8：占⽤8位，范围约±10⁶，精度约2-3位有效数字  

优势在于大幅度减少了内存占用，提升计算效率的同时保证了一定程度的精度，成本下降。

**混合精度训练：**  
NVIDIA H100 GPU 支持FP8 Tensor Core，计算效率显著提升，混合精度训练采用分层策略，对精度不敏感用FP8，反之用FP32。  
分层策略：  
Embedding层用FP8：嵌入层对精度要求相对较低  
Transformer主体用BF16：核心计算保持较高精度    
LayerNorm用FP32：归一化层需要最高精度保证数值稳定性  

### （5）MLA多头潜在注意力机制的讲解

主要用于解决KV缓存优化，计算效率提升，显存开销瓶颈三个问题。有三大核心技术：低秩联合压缩，矩阵吸收，位置编码解耦。

#### （一）低秩联合压缩  
1. 背景与动机

- **问题**：标准 Transformer 的 KV Cache 显存为 $ O(n d H) $（$ n $=上下文长度，$ d $=隐藏维，$ H $=head 数）。
- 当 $ n > 32K $ 时，单卡显存无法承载。
- **现有方案不足**：
  - MQA/GQA：缓存仍 $ O(n) $；
  - KV 剪枝/量化：有精度损失；
  - 简单降维：缓存仍随 $ n $ 增长。
- **MLA 目标**：**让 KV Cache 大小与 $ n $ 无关**，同时保持多头表达能力与长上下文建模能力。



2. 核心思想

> 人类不会记住对话的每个字，而是提炼关键要点。  
> MLA 用 **$ L $ 个可学习的 latent 向量**（如 $ L=8 $，长度为r）作为整个上下文的“记忆摘要”，所有 future tokens 都通过查询这 $ L $ 个 latent 来获取历史信息。



3. 整体架构

MLA 模块替换标准 Multi-Head Attention（MHA），包含两个子模块：

| 模块 | 功能 |
|------|------|
| **Latent Compression Module** | 将当前上下文压缩为 $ Z \in \mathbb{R}^{L \times r} $ |
| **Latent Query Module** | 每个 head 从 $ Z $ 生成自己的 K/V，并计算注意力 |

---

4. 详细流程

（1）压缩阶段（Compression）—— 构建记忆

当处理 token 序列 $ X = [x_1, ..., x_m] \in \mathbb{R}^{m \times d} $ 时：

**投影到低维空间**：
   $$
   H = X W_c \in \mathbb{R}^{m \times r}, \quad W_c \in \mathbb{R}^{d \times r}
   $$
   （$ r $ 通常为 128） $W_c$通常包含多个操作，这里用到了矩阵吸收技术  

**用 $ L $ 个可学习 latent queries 做 cross-attention**：
   - 初始化 **latent queries**：$ Q_{\text{latent}} \in \mathbb{R}^{L \times r} $（模型参数，训练时学习）
   - 计算：
     $$
     Z = \text{Softmax}\left( \frac{Q_{\text{latent}} H^\top}{\sqrt{r}} \right) H \in \mathbb{R}^{L \times r}
     $$
   - **$ Z $ 即为压缩后的 KV Cache**，显存 = $ L \times r $（如 8×128=1024 floats）

> ✅ **关键**：$ Z $ 与 $ m $ 无关！无论上下文多长，只存 8 个向量。

（2）查询阶段（Query）—— 读取记忆

当新 token $ x_{\text{new}} $ 到来：

计算其 query：
   $$
   q = x_{\text{new}} W_q \in \mathbb{R}^{d}
   $$

对第 $ h $ 个 attention head：
   - 使用**专属上投影矩阵**生成 K/V 表示：
     $$
     K^{(h)} = Z W_k^{(h)\top} \in \mathbb{R}^{L \times d}, \quad
     V^{(h)} = Z W_v^{(h)\top} \in \mathbb{R}^{L \times d}
     $$
     （$ W_k^{(h)}, W_v^{(h)} \in \mathbb{R}^{r \times d} $ 为可学习参数）$ W_k^{(h)}$也由多个部分组成，此处用到了矩阵吸收技术  

**计算注意力分数**（结合 RoPE）：
   - MLA **解耦 RoPE**：位置信息仅加在 query 上，K/V 的 latent 与位置无关；
   - 实际计算：
     $$
     \text{Attn}^{(h)} = \text{Softmax}\left( \frac{(q \circ \text{RoPE}(t)) (K^{(h)})^\top}{\sqrt{d}} \right)
     $$
     （$ t $ 为当前位置）

**计算输出**：
   $$
   \text{Out}^{(h)} = \text{Attn}^{(h)} V^{(h)} \in \mathbb{R}^{d}
   $$

多头拼接 + 投影：
   $$
   \text{Output} = \text{Concat}(\text{Out}^{(1)}, ..., \text{Out}^{(H)}) W_o
   $$

---

5. 关键技术亮点

| 亮点 | 说明 |
|------|------|
| **KV Cache 与序列长度解耦** | 缓存大小 = $ L \times r $（常数），支持 128K+ 上下文 |
| **多头多样性保留** | 共享 $ Z $，但每个 head 有独立 $ W_k^{(h)}, W_v^{(h)} $ |
| **RoPE 解耦设计** | 位置信息仅加在 query，latent 学习内容语义 |
| **端到端可训练** | latent queries 和投影矩阵全部可学习 |

---

6. 训练 vs 推理

| 阶段 | 操作 |
|------|------|
| **训练** | 对每个 batch，动态计算 $ Z $ 并参与 loss，梯度回传更新所有参数 |
| **推理** | - 首次处理 token 时计算 $ Z $ 并缓存<br>- 后续 token 直接读取 $ Z $，无需重算<br>- $ Z $ 可增量更新（工程优化） |

> ⚡ 推理时，KV Cache 显存恒定，延迟稳定。

---

7. 性能与效果（来自 DeepSeek-V2 技术报告）

| 指标 | MLA vs MHA |
|------|-----------|
| KV Cache 显存 | **↓ 97%+**（32K 上下文下，从 320MB → 1KB） |
| 推理速度 | ↑ 2–3 倍 |
| 长上下文任务（如 RULER） | **性能持平甚至略优** |
| API 成本 | ↓ 37%（金融风控案例） |

---

8. 与常见误解的根本区别

| 常见误解 | MLA 真实机制 |
|----------|------------|
| 对每个 token 的 K/V 降维 | 对**整个上下文**做摘要，生成 $ L $ 个 latent |
| 缓存大小仍 $ O(n) $ | 缓存大小 $ O(1) $（与 $ n $ 无关） |
| “还原” = 投影回去 | **不还原原始 K/V**，而是生成 head-specific 表示 |
| 信息易丢失 | 通过端到端训练，latent 自动保留关键信息 |

**总结：**  
**MLA 不是“压缩 KV”，而是“重构注意力的记忆机制”**。  
它将 KV Cache 从“逐 token 存储”升级为“全局语义摘要”，  
实现了 **显存常数化、计算轻量化、性能无损化** 的三重突破，  
是支撑 DeepSeek-V2 以极低成本提供 128K 上下文服务的核心技术。

# 矩阵吸收与位置编码解耦技术详解

## 一、背景：为什么需要这两项技术？

在高效注意力机制（如 MLA、低秩 KV 压缩）中，直接对原始 Key/Value 矩阵进行压缩或变换会带来两个关键问题：

1. **信息损失**：压缩过程可能丢弃对注意力计算至关重要的动态范围或方向信息；
2. **位置编码冲突**：传统 RoPE（Rotary Position Embedding）作用于 K/V 向量，但压缩后 K/V 不再显式存在，导致位置信息丢失。

为解决这些问题，DeepSeek-V2 等模型引入了两项关键技术：
- **矩阵吸收**（Matrix Absorption）：将线性变换“吸收”进投影权重，避免中间表示失真；
- **位置编码解耦**（Positional Encoding Disentanglement）：将位置信息仅作用于 Query，使 K/V 可安全压缩。

---

## 二、矩阵吸收（Matrix Absorption）

### 1. 问题起源

在标准注意力中：
$$
K = X W_K,\quad V = X W_V
$$
若对 $K, V$ 做后处理（如降维、量化、低秩近似），相当于：
$$
K' = K A = X (W_K A)
$$
其中 $A$ 是压缩/变换矩阵。  
但若在推理时**先算 $K = X W_K$，再乘 $A$**，会引入额外计算和数值误差。

### 2. 核心思想

> **将变换矩阵 $A$ 吸收到原始投影权重中，形成新的等效权重**，从而在前向传播中**一步到位**，无需中间变量。

即：
$$
W_K^{\text{absorbed}} = W_K A \quad \Rightarrow \quad K' = X W_K^{\text{absorbed}}
$$

### 3. 在 MLA 中的应用

- 在压缩阶段，$H = X W_c$；
- 若后续还有线性变换（如 LayerNorm、scaling），可将其**吸收进 $W_c$**；
- 在升维阶段，$K^{(h)} = Z W_k^{(h)\top}$，若 $W_k^{(h)}$ 包含固定变换（如转置、缩放），也可预先吸收。

### 4. 优势

| 优势 | 说明 |
|------|------|
| **减少计算步骤** | 避免中间张量生成，提升推理速度 |
| **数值稳定性** | 减少浮点误差累积 |
| **便于量化/编译优化** | 权重融合后更易部署到 TensorRT、vLLM 等推理引擎 |

> ✅ **本质**：矩阵吸收是一种**计算图融合优化**，属于“训练-推理一致性”保障手段。

---

## 三、位置编码解耦（Positional Encoding Disentanglement）

### 1. 传统 RoPE 的问题

标准 RoPE 定义为：
$$
\text{RoPE}(x, t) = R_t x
$$
其中 $R_t$ 是旋转矩阵，$t$ 是位置。

在 MHA 中，通常对 Q 和 K 同时加 RoPE：
$$
Q = \text{RoPE}(X W_Q, t),\quad K = \text{RoPE}(X W_K, t')
$$
这样注意力分数为：
$$
Q K^\top = (R_t X W_Q)(R_{t'} X W_K)^\top = X W_Q W_K^\top X^\top \cdot \cos(\theta_{t - t'})
$$
→ **位置信息通过 Q 与 K 的相对旋转体现**。

### 2. 压缩场景下的冲突

在 MLA 等机制中：
- **K 不再是每个 token 的向量**，而是从 latent $Z$ 生成的 $L \times d$ 矩阵；
- 若对 $K$ 应用 RoPE，需知道每个 latent 对应的“位置”，但 latent 是全局摘要，**无明确位置**！

强行加 RoPE 会导致：
- 位置语义混乱；
- 长上下文泛化能力下降。

### 3. 解耦方案：仅对 Query 加 RoPE

> **将位置信息完全交给 Query，K/V 仅保留内容语义**。

具体做法：
- **Query**：$ q = \text{RoPE}(x_{\text{new}} W_q, t) $
- **Key**：$ K^{(h)} = Z W_k^{(h)\top} $（**无 RoPE**）
- 注意力计算：
  $$
  \text{Attn} = \text{Softmax}\left( \frac{q (K^{(h)})^\top}{\sqrt{d}} \right)
  $$

### 4. 为什么可行？

- 实验发现：**只要 Query 携带位置信息，模型仍能学习到合理的相对位置偏好**；
- Latent $Z$ 学习的是“哪些内容重要”，而“何时出现”由 Query 的 RoPE 动态调制；
- 在训练中，模型自动学会将位置敏感性编码到 $W_k^{(h)}$ 和 $Z$ 的交互中。

### 5. 效果验证（DeepSeek-V2）

| 方法 | RULER 准确率 | 128K 上下文稳定性 |
|------|--------------|------------------|
| 标准 RoPE（Q+K） | 74.8% | ❌ 无法支持（KV 无法压缩） |
| RoPE 仅用于 Q（解耦） | **73.5%** | ✅ 稳定支持 128K |
| 无 RoPE | 62.1% | ✅ 但性能崩坏 |

> 📌 **结论**：位置编码解耦是 MLA 能同时实现**长上下文支持 + 高精度**的关键前提。

---

## 四、两项技术的协同作用

| 技术 | 解决的问题 | 协同效果 |
|------|-----------|--------|
| **矩阵吸收** | 避免压缩/升维过程中的计算冗余与数值失真 | 使 latent 投影更精确 |
| **位置编码解耦** | 允许 K/V 被安全压缩而不丢失位置语义 | 使 MLA 架构在理论上成立 |

> 🔗 **二者共同保障了 MLA 的“端到端可训练性”与“推理高效性”**。

---

## 五、总结

- **矩阵吸收** 是一种**计算优化技术**，通过权重融合减少中间步骤，提升效率与稳定性；
- **位置编码解耦** 是一种**架构设计创新**，将位置信息与内容表示分离，使 KV 压缩成为可能；
- 在 DeepSeek-V2 的 MLA 中，**两者缺一不可**：
  - 没有解耦 → 无法压缩 K/V；
  - 没有吸收 → 推理慢、精度低。

> 💡 **最终效果**：  
> 模型既能以 **1KB 级缓存**处理 128K 上下文，又能保持 **接近原始 MHA 的性能**，  
> 实现了**效率与能力的统一**。

### （6）长度外推方案

定义：模型训练时使用较短序列，但推理时处理长序列

为什么需要长度外推：因为长序列训练消耗高，但实际应用又常常是长序列，GPU内存限制了训练长度。

**YARN（Yet Another RoPE Extension）：**  
DeepSeek的外推改进方案，基于RoPE的改进。  
以前笔记有提到过。  
YARN的优势在于：支持比RoPE更长的外推序列，扩展后模型性能不下降，也不需要重新训练长序列模型，适用于各种基于RoPE的模型。

### （7）GRPO优化算法（Group Relative Policy Optimization）

定义：这是deepseek团队提出的一种RLHF的推理优化算法，专门针对优化大模型的推理能力，核心在于按步骤得分机制和组内相对优势计算，显著提升模型推理性能。

设计背景：  
传统强化学习算法存在只对结果评分，忽略了推理过程的重要性，难以引导模型学会推理过程，且容易发生梯度消失和梯度爆炸。  
GRPO的核心创新点在于：  
将多个模型生成的推理路径（如 CoT）组成一个“组”（group）；  
在组内进行相对比较（而非依赖绝对奖励），通过排序或胜率来构造优势（advantage）信号；  
利用组内样本之间的相对优劣关系来更新策略，从而避免训练单独的奖励模型；  
支持端到端训练，并能有效利用多步推理轨迹中的信息。  

## 二、DeepSeek-V3.2EXP的技术更新

### (1)稀疏注意力机制DSA

原理是像人类一样选择性关注重要token而不是处理所有token之间的联系

DSA的核心组件是：
1、闪电索引器：